# Introduction

This custom data validation tool was created for a specific client to validate their incoming data. 

The purpose of this validator is to highlight the errors inside the provided by the client data that will prevent its use.

This data validator does not fix any issues inside the data since it's not the purpose of this tool. 


# Table of Content
- [Introduction](#Introduction)
- [Library Imports](#Imports)
- [Data Upload](#data-loading)
- [Technical Validation](#Technical-Validation).
  - [Catalog](#Catalog)
    - [Catalog Mandatory Columns](#Catalog-Mandatory-Columns)
    - [Catalog Data type check](#Catalog-Data-type-check)
    - [Catalog Missing values check](#Catalog-Missing-values-check)
    - [Catalog Numeric values range check](#Catalog-Numeric-values-range-check)
  - [Inventory](#Inventory)
    - [Inventory Mandatory Columns](#Inventory-Mandatory-Columns)
    - [Inventory Data type check](#Inventory-Data-type-check)
    - [Inventory Missing values check](#Inventory-Missing-values-check)
    - [Inventory Numeric values range check](#Inventory-Numeric-values-range-check)

# Library Imports

In [1]:
import pandas as pd

In [2]:
#Set color
red_color = '\033[91m'
green_color = '\033[92m'
reset_color = '\033[0m'

---

# Data Upload

In the data folder should exist 4 files:

catalog - a file that contains all information about the inventory

inventory - a file that contains the current inventory status of location-inventory

location - a file that contains all information about the location

transactions - a file that contains all information about all transactions (location-inventory based)

Every file should contain a specific set of mandatory collumns, other wise the data validator will stop at the stage of data upload.


In [3]:
#Name of the input files
catalog_file_name = 'catalog.csv'
inventory_file_name = 'inventory.csv'
locations_file_name = 'locations.csv'
transactions_file_name = 'transactions.csv'

In [4]:
catalog_file_name

df_catalog = pd.read_csv('data/' + catalog_file_name,
                        dtype={
                                'id': 'str',
                                'name': 'str', 
                                'product_id': 'str'
                            })
df_inventory = pd.read_csv('data/' + inventory_file_name, 
                            dtype={
                                'location_id': 'str', 
                                'sku_id': 'str',
                                'source_location_id':'str'
                             })
df_locations = pd.read_csv('data/' + locations_file_name, 
                            dtype={
                                'id': 'str',
                                'name': 'str',
                                'type': 'str'
                             })
df_transactions = pd.read_csv('data/' + transactions_file_name, 
                               dtype={
                                   'location_id': 'str', 
                                   'sku_id': 'str',
                                   'source_location_id':'str',
                                   'target_location_id': 'str'
                                 })

---

# Technical Validation

## Catalog

### Catalog Mandatory Columns

Checking that all the mandator columns are present, columns - **[id,name,product_id,price,cost]**

In [5]:
catalog_column_list = ['id','name','product_id','price','cost']

In [6]:
for column in catalog_column_list:
    assert column in df_catalog.columns, f'{column} column is missing from the catalog file' 
print(green_color + 'All columns are present.' + reset_color)

All columns are present.


### Catalog Data type check

Chacks if the column can be transformed in to the data type that the sustem expects

In [7]:
#List of Dtypes of mandator columns
catalog_columns_dtype = {'id':'str','name':'str','product_id':'str','price':'float','cost':'float'}

In [8]:
df_catalog['id'].dtype 

dtype('O')

In [9]:
for column, dtype in catalog_columns_dtype.items():
    if df_catalog[column].dtype == 'object' and dtype == 'str':
        try:
            df_catalog[column] = df_catalog[column].astype('str')
        except ValueError:
            raise ValueError(red_color + f"Unable to change dtype of {column} column from object to str" + reset_color)
    else:
        try:
            pd.to_numeric(df_catalog[column], errors='raise', downcast=dtype)
        except (TypeError, ValueError):
            print(red_color + f"{column} column has dtype mismatch with {dtype}" + reset_color)

### Catalog Missing values check

Checks if the column contains any null values.

In [10]:
for column in catalog_column_list:
    try:
        assert df_catalog[column].isnull().sum().sum() == 0
        print(green_color + f'{column} column is populated with data' + reset_color)
    except:
        print(red_color + f'There are null values in the {column}.' + reset_color)

id column is populated with data
name column is populated with data
product_id column is populated with data
price column is populated with data
cost column is populated with data


### Catalog Numeric values range check

Checks if the values in the column are in expected range.

In [11]:
try:
    assert (df_catalog['price'] >= 0).all()
    print(green_color + 'All is good, Price is >= 0' + reset_color)
except AssertionError:
    print(red_color + 'Price contains values less than 0' + reset_color)
    total_count = len(df_catalog['price'])
    negative_count = (df_catalog['price'] < 0).sum()
    percentage = (negative_count / total_count) * 100
    print(red_color + f"Percentage of price values less than 0: {percentage}%" + reset_color)

try:
    assert np.issubdtype(df_catalog['price'].dtype, np.floating) or np.issubdtype(df_catalog['price'].dtype, np.integer)
    print(green_color + 'All is good, delimiters in the Price column are in expected range' + red_color)
except:
    print(red_color + 'The Price column contains non-numeric values' + reset_color)

All is good, Price is >= 0
The Price column contains non-numeric values


In [12]:
try:
    assert (df_catalog['cost'] >= 0).all()
    print(green_color + 'All is good, Cost is >= then 0' + reset_color)
except:
    print(red_color + 'Cost contains values less than 0' + reset_color)
    total_count = len(df_catalog['cost'])
    negative_count = (df_catalog['cost'] < 0).sum()
    percentage = (negative_count / total_count) * 100
    print(red_color + f"Percentage of cost values less than 0: {percentage}%" + reset_color)

try:
    assert np.issubdtype(df_catalog['cost'].dtype, np.floating) or np.issubdtype(df_catalog['cost'].dtype, np.integer)
    print(green_color + 'All is good, delimiters in the Cost column are in expected range' + reset_color)
except:
    print(red_color + 'The Cost column contains non-numeric values' + reset_color)

All is good, Cost is >= then 0
The Cost column contains non-numeric values


## Inventory

### Inventory Mandatory Columns

Checking that all the mandator columns are present, columns - **[location_id,sku_id,source_location_id,transit_qty,site_qty,status_date]**

In [13]:
#columns list
inventory_column_list = ['location_id','sku_id','source_location_id','transit_qty','site_qty','status_date']

In [14]:
for column in inventory_column_list:
    try:
        assert column in df_inventory.columns
        print(green_color + f'{column} column is present.' + reset_color)
    except:
        print(red_color + f'{column} column is missing from the inventory file'  + reset_color)

location_id column is present.
sku_id column is present.
source_location_id column is present.
transit_qty column is present.
site_qty column is present.
status_date column is present.


### Inventory Data type check

Checks if the column can be transformed into the data type that the system expects

In [15]:
#List of Dtypes of mandator columns
inventory_columns_dtype = {'location_id':'str',
                           'sku_id':'str',
                           'source_location_id':'uuid',
                           'transit_qty':'float',
                           'site_qty':'float',
                           'status_date':'timestamp'}

In [16]:
for column, dtype in inventory_columns_dtype.items():
    if dtype == 'timestamp':
        try:
            pd.to_datetime(df_inventory[column], format='ISO8601', errors='raise')
        except (TypeError, ValueError):
            print(red_color + f"{column} column has dtype mismatch with {dtype}" + reset_color)
    elif df_inventory[column].dtype == 'object' and dtype == 'str':
        try:
            df_inventory[column] = df_inventory[column].astype('str')
        except ValueError:
            print(red_color + f"Unable to change dtype of {column} column from object to str" + reset_color)
    elif df_inventory[column].dtype == 'int' and dtype == 'float':
        try:
            df_inventory[column] = df_inventory[column].astype('float')
        except ValueError:
            print(red_color + f"Unable to change dtype of {column} column from object to str" + reset_color)
    elif dtype == 'uuid':
        grouped = df_inventory.groupby(['status_date', 'sku_id', 'location_id'])['source_location_id'].nunique()
        if (grouped > 1).any():
            print(red_color + "Multiple unique source_location_id found for the same status_date, sku_id, and location_id." + reset_color)
        else:
            print(green_color + "Each status_date, sku_id, and location_id has only one unique source_location_id." + reset_color)
        
print(green_color + 'All dtypes are matching the requirements' + reset_color)

Each status_date, sku_id, and location_id has only one unique source_location_id.
status_date column has dtype mismatch with timestamp
All dtypes are matching the requirements


### Inventory Missing values check

Checks if the column contains any null values.

In [17]:
for column in inventory_column_list:
    try:
        assert df_inventory[column].isnull().sum().sum() == 0
        print(green_color + f'{column} column is populated with data' + reset_color)
    except:
        print(red_color + f'There are null values in the {column}.' + reset_color)

location_id column is populated with data
sku_id column is populated with data
source_location_id column is populated with data
transit_qty column is populated with data
site_qty column is populated with data
status_date column is populated with data


### Inventory Numeric values range check

Checks if the values in the column are in the expected range. transit_qty and site_qty can be less then 0

In [18]:
try:
    assert df_inventory['transit_qty'].min() >= 0
    print(green_color + f'There is no negative transit_qty' + reset_color)
except:
    print(red_color + f'There is negative transit_qty for some inventories' + reset_color)
    print(df_inventory[df_inventory['transit_qty']<0])

There is no negative transit_qty


## Locations

Checking that all the mandator columns are present, columns - [id,name,type]

In [19]:
locations_column_list = ['id','name','type']

In [20]:
for column in locations_column_list:
    try:
        assert column in df_locations.columns
        print(green_color + f'{column} column is present.' + reset_color)
    except:
        print(red_color + f'{column} column is missing from the locations file' + reset_color)

id column is present.
name column is present.
type column is present.


### Dtype check

Chacks if the column can be transformed in to the data type that the sustem expects

In [21]:
#List of Dtypes of mandator columns
locations_columns_dtype = {'id':'str',
                           'name':'str'}

In [22]:
for column, dtype in locations_columns_dtype.items():
    if df_locations[column].dtype == 'object' and dtype == 'str':
        try:
            df_locations[column] = df_locations[column].astype('str')
            print(green_color + f'{column} column dtype is matching the requirements' + reset_color)
        except ValueError:
            print(red_color + f"Unable to change dtype of {column} column from object to str" + reset_color)

id column dtype is matching the requirements
name column dtype is matching the requirements


### Missing values check

Check if the column contains any null values.

In [23]:
for column in locations_column_list:
    try:
        assert df_locations[column].isnull().sum().sum() == 0
        print(green_color + f'{column} column is populated with data' + reset_color)
    except:
        print(red_color + f'There are null values in the {column}.' + reset_color)

id column is populated with data
name column is populated with data
type column is populated with data


### Enum validation

Checks if the location types are in expected range of values. Expected types: ['store', 'warehouse', 'vwarehouse','ecommerce', 'plant', 'supplier','outlet', 'unknown', 'client']

In [24]:
locations_type_enum = ['store', 'warehouse', 'vwarehouse','ecommerce', 'plant', 'supplier', 'outlet', 'unknown', 'client']
try:
    assert df_locations['type'].isin(locations_type_enum).all()
    print(green_color + 'All location types are correct' + reset_color)
except:
    invalid_types = df_locations[~df_locations['type'].isin(locations_type_enum)]
    print(red_color + f"Wrong location type in column: {column}" + reset_color)
    print(red_color + f"Invalid location types: {invalid_types['type'].unique()}" + reset_color)

Wrong location type in column: type
Invalid location types: ['Shop' 'Warehouse' 'Warehouse ']


### Transactions

Checking that all the mandator columns are present, columns - ['id','sku_id','source_location_id','target_location_id','quantity','type','transaction_date']

In [25]:
transactions_column_list = ['id','sku_id','source_location_id','target_location_id','quantity',
                            'type','transaction_date']

In [26]:
for column in transactions_column_list:
    try:
        assert column in df_transactions.columns
        print(green_color + f'{column} column is present.' + reset_color)
    except:
        print(red_color +  f'{column} column is missing from the transactions file' + reset_color)

id column is present.
sku_id column is present.
source_location_id column is present.
target_location_id column is present.
quantity column is present.
type column is present.
transaction_date column is present.


### Dtype check

Chacks if the column can be transformed in to the data type that the sustem expects

In [27]:
transactions_columns_dtype = {'id':'str','sku_id':'str','source_location_id':'str','target_location_id':'str','quantity':'float','type':'str','transaction_date':'timestamp'}

In [28]:
for column, dtype in transactions_columns_dtype.items():
    if dtype == 'timestamp':
        try:
            pd.to_datetime(df_transactions[column], format='ISO8601', errors='raise')
        except:
            print(red_color + f"{column} column has dtype mismatch with {dtype}" + reset_color)
    elif df_transactions[column].dtype == 'object' and dtype == 'str':
        try:
            df_transactions[column] = df_transactions[column].astype('str')
        except:
            print(red_color + f"Unable to change dtype of {column} column from object to str" + reset_color)
    elif df_transactions[column].dtype == 'int' and dtype == 'float':
        try:
            df_transactions[column] = df_transactions[column].astype('float')
        except:
            print(red_color + f"Unable to change dtype of {column} column from object to str" + reset_color)
        
print(green_color + 'All dtypes are matching the requirements' + red_color)

transaction_date column has dtype mismatch with timestamp
All dtypes are matching the requirements


### Missing values check

Checks if the column contains any null values.

In [29]:
for column in transactions_columns_dtype:
    try:
        assert df_transactions[column].isnull().sum().sum() == 0
        print(green_color + f'{column} column is populated with data' + reset_color)
    except: 
        print(red_color +  f'There are null values in the {column}.' + reset_color)

id column is populated with data
sku_id column is populated with data
source_location_id column is populated with data
target_location_id column is populated with data
There are null values in the quantity.
type column is populated with data
transaction_date column is populated with data


### Enum validation

Checks if the transactions types are in expected range of values. Expected types: **['in', 'out', 'sale']**

In [30]:
transactions_type_enum = ['in','out','sale']
try:
    assert df_transactions['type'].isin(transactions_type_enum).all()
    print(green_color + 'All transaction types are correct' + reset_color)
except:
    false_types = df_transactions.loc[~df_transactions['type'].isin(transactions_type_enum), 'type']
    print(red_color + 'Wrong transaction types:' + reset_color)
    print(false_types)

Wrong transaction types:
0            IN
1            IN
2            IN
3            IN
4            IN
           ... 
1048570    SALE
1048571    SALE
1048572    SALE
1048573    SALE
1048574    SALE
Name: type, Length: 1048575, dtype: object


### Numeric values range check

Checks if the values in the column are in expected range. quantity can't be less then 0

In [31]:
try:
    assert (df_transactions['quantity'] >= 0).all()
    print(green_color + 'All is good, quantity is >= then 0' + reset_color)
except:
    print(red_color + 'quantity contains values less than 0' + reset_color)
    total_count = len(df_transactions['quantity'])
    negative_count = (df_transactions['quantity'] < 0).sum()
    percentage = (negative_count / total_count) * 100
    print(red_color + f"Percentage of quantity values less than 0: {percentage}%" + reset_color)

quantity contains values less than 0
Percentage of quantity values less than 0: 1.7239587058627184%


# Logical Validation
---

## Catalog Logical Validation


### SKU ID is a unique value across all rows

In [32]:
try:
    assert not df_catalog['id'].duplicated().any()
    print(green_color + 'No ID duplication found in the Catalog' + reset_color)
except:
    print(red_color + "Duplicate IDs found in the 'id' column" + reset_color)

Duplicate IDs found in the 'id' column


### For each SKU, at least price or cost should be bigger than 0

In [33]:
invalid_rows = df_catalog[(df_catalog['price'] == 0) & (df_catalog['cost'] == 0)][['id', 'cost', 'price']]
total_skus = len(df_catalog)
invalid_sku_percentage = len(invalid_rows) / total_skus * 100

if len(invalid_rows) > 0:
    print(red_color + "Invalid price or cost values in the following examples:" + reset_color)
    print(invalid_rows.head(5))
    print(red_color + f"Percentage of SKUs with invalid price or cost values: {invalid_sku_percentage:.2f}%" + reset_color)
else:
    print(green_color + 'Price or Cost is present for each SKU' + reset_color)


Price or Cost is present for each SKU


### All SKUs of a product should have the same categories

In [34]:
# Check if 'colors' column is present
if 'colors' not in df_catalog.columns:
    print(red_color + "Column 'colors' is not presented." + reset_color)
else:
    # Perform the logical validation
    grouped = df_catalog.groupby('product_id')['categories'].nunique()
    invalid_ids = grouped[grouped > 1].index

    if len(invalid_ids) > 0:
        print(red_color + "Non mandatory column validation" + reset_color)
        print(red_color + "Following products have more than 1 category:" + reset_color)
        print(invalid_ids)
    else:
        print(green_color + 'All SKUs of a product have same categories' + reset_color)

Column 'colors' is not presented.


### All SKUs of a product should have the same color

In [35]:
# Check if 'colors' column is present
if 'colors' not in df_catalog.columns:
    print(red_color + "Column 'colors' is not presented." + reset_color)
else:
    # Perform the logical validation
    grouped = df_catalog.groupby('product_id')['colors'].nunique()
    invalid_ids = grouped[grouped > 1].index

    if len(invalid_ids) > 0:
        print(red_color + "Non mandatory column validation" + reset_color)
        print(red_color + "Inconsistent colors for the following first 5 product IDs:" + reset_color)
        print(df_catalog[df_catalog['product_id'].isin(invalid_ids)][['product_id', 'id', 'colors']].head(5))
    else:
        print(green_color + 'Same color present for all of the SKUs of the same product' + reset_color)

Column 'colors' is not presented.


### Number of unique sizes in a product should be equal to the number of SKUs

In [36]:
# Check if 'size' column is present
if 'size' not in df_catalog.columns:
    print(red_color + "Column 'size' is not presented." + reset_color)
else:
    # Perform the logical validation
    grouped = df_catalog.groupby('product_id')['size'].nunique()
    sku_counts = df_catalog.groupby('product_id')['id'].nunique()

    invalid_ids = grouped[grouped != sku_counts].index

    if len(invalid_ids) > 0:
        print(red_color + "Non mandatory column validation" + reset_color)
        print(red_color + "Mismatch in the number of unique sizes and SKUs for the following product IDs:" + reset_color)
        print(df_catalog[df_catalog['product_id'].isin(invalid_ids)][['id', 'product_id', 'size']])
    else:
        print(green_color + 'Same size present for all of the SKUs of the same product' + reset_color)


Column 'size' is not presented.


### At least 1 SKU is not on avoid replenishment

In [37]:
try:
    assert (df_catalog['avoid_replenishment'] == 1).all()
    print(red_color + "Non mandatory column validation" + reset_color)
    print(red_color + 'All inventories are on avoid replenishment' + reset_color)
except:
    print(green_color + 'At least 1 inventory is not on avoid replenishment' + reset_color)

At least 1 inventory is not on avoid replenishment


## Inventory Logical Validation

### At least 1 inventory is not on avoid replenishment

In [38]:
try:
    assert (df_inventory['avoid_replenishment'] == 0).any()
    print(red_color + "Non mandatory column validation" + reset_color)
    print(red_color + 'All inventories are on avoid replenishment' + reset_color)
except:
    print(green_color + 'At least 1 inventory is not on avoid replenishment' + reset_color)

At least 1 inventory is not on avoid replenishment


### All the Locations exist in the DB or input files

In [39]:
try:
    assert df_inventory['location_id'].isin(df_locations['id']).all()
    print(green_color + 'All locations from the Inventory are present in the Locations file' + reset_color)
except:
    print(red_color + "Invalid location IDs found in df_inventory" + reset_color)
    print(f"Missing Locations:\n{df_inventory.loc[~(df_inventory['location_id'].isin(df_locations['id'])), 'location_id'].unique()}")

Invalid location IDs found in df_inventory
Missing Locations:
['000' '001' '002' '003' '004' '005' '006' '007' '008' '009' '010' '011'
 '012' '013' '014' '016' '017' '018' '019' '020' '021' '022' '023' '025'
 '026' '027' '028' '029' '033' '034' '035' '036' '037' '038' '039' '040'
 '041' '042' '043']


### All the SKUs exist in the Catalog

In [40]:
invalid_sku_ids = df_inventory[~df_inventory['sku_id'].isin(df_catalog['id'])]['sku_id'].unique()

if invalid_sku_ids.any():
    print(red_color + "Invalid SKU IDs found in df_inventory:" + reset_color)
    print(invalid_sku_ids)

    invalid_sku_count = len(invalid_sku_ids)
    total_sku_count = len(df_inventory['sku_id'].unique())

    print(red_color + 'Total SKUs: {}'.format(total_sku_count) + reset_color)
    print(red_color + 'Number of invalid SKUs: {}'.format(invalid_sku_count) + reset_color)

    invalid_sku_percentage = (invalid_sku_count / total_sku_count) * 100
    print(red_color + 'Percentage of invalid SKUs from total SKUs: {:.2f}%'.format(invalid_sku_percentage) + reset_color)

else:
    print(green_color + "All SKU IDs in df_inventory exist in the df_catalog" + reset_color)

Invalid SKU IDs found in df_inventory:
['725887-815' '894081-463' '894331-463' ... 'M00107000024' 'M00112000003'
 'YFM19CPT']
Total SKUs: 33468
Number of invalid SKUs: 3733
Percentage of invalid SKUs from total SKUs: 11.15%


## Locations Logical Validation

### Location ID is a unique value across all rows

In [41]:
try:
    assert not df_locations['id'].duplicated().any()
    print(green_color + 'No ID duplication found in the Locations file' + reset_color)
except:
    print(red_color + "Duplicate IDs found in the 'id' column" + reset_color)

No ID duplication found in the Locations file


### There is at least 1 location from type “store”

In [42]:
try:
    assert (df_locations['type'] == 'store').any()
    print(green_color + "At least one location of type 'store' exists" + reset_color)
except:
    print(red_color + "No locations of type 'store' found" + reset_color)

No locations of type 'store' found


### There is at least 1 location from type “warehouse”

In [43]:
try:
    assert (df_locations['type'] == 'warehouse').any()
    print(green_color + "At least one location of type 'warehouse' exists" + reset_color)
except:
    print(red_color + "No locations of type 'warehouse' found" + reset_color)

No locations of type 'warehouse' found


### All the location types are from the optional values list (wh/store/plant/supplier/client/unknown)

In [44]:
optional_values = {'warehouse', 'store', 'plant', 'supplier', 'client', 'unknown','outlet','ecommerce'}
location_types = set(df_locations['type'].unique())
invalid_types = location_types - optional_values

try:
    assert location_types.issubset(optional_values)
    print(green_color + "All location types are from the optional values list" + reset_color)
except:
    if invalid_types:
        print(red_color + "Invalid location types found:" + reset_color)
        print(invalid_types)

Invalid location types found:
{'Warehouse', 'Shop', 'Warehouse '}


### At least 1 location is not on avoid replenishment

In [45]:
try:
    assert (df_locations['avoid_replenishment'] == 0).any()
    print(green_color + 'At least 1 location is not on avoid replenishment' + reset_color)
except: 
    print(red_color + "Non mandatory column validation" + reset_color)
    print(red_color + 'All locations are on avoid replenishment' + reset_color)

Non mandatory column validation
All locations are on avoid replenishment


## Transactions Logical Validation

### Transaction ID is a unique value across all rows

In [46]:
try:
    assert not df_transactions['id'].duplicated().any()
    print(green_color + "Transaction ID is unique across all rows" + reset_color)
except:
    print(red_color + "Transaction ID is not unique" + reset_color)

Transaction ID is unique across all rows


### Source location is different from the target location

In [47]:
try:
    assert (df_transactions['source_location_id'].ne(df_transactions['target_location_id'])).all()
    print(green_color + "Source location is different from the target location in all rows" + reset_color)
except:
    print(red_color + "Source location is the same as the target location" + reset_color)

Source location is different from the target location in all rows


### Source location OR Target location exist as a known location

In [48]:
known_location_ids = set(df_locations['id'].unique())
invalid_locations = ~(df_transactions['source_location_id'].isin(known_location_ids) |
                     df_transactions['target_location_id'].isin(known_location_ids))

invalid_source_locations = df_transactions.loc[invalid_locations, 'source_location_id']
invalid_target_locations = df_transactions.loc[invalid_locations, 'target_location_id']

if invalid_locations.any():
    if invalid_source_locations.any():
        print(red_color + "Invalid source locations:" + reset_color)
        print(invalid_source_locations)
    if invalid_target_locations.any():
        print(red_color + "Invalid target locations:" + reset_color)
        print(invalid_target_locations)
else:
    print(green_color + "All source and target locations exist as known locations" + reset_color)

Invalid source locations:
81802     34
81803     34
81804     34
81805     34
81806     34
          ..
442329     0
442330     0
442331     0
450402     0
450403     0
Name: source_location_id, Length: 292, dtype: object
Invalid target locations:
81802      82303
81803      82304
81804      82305
81805      82306
81806      82307
           ...  
442329    442830
442330    442831
442331    442832
450402    450903
450403    450904
Name: target_location_id, Length: 292, dtype: object


### All SKU_ids exist in the catalog

In [49]:
invalid_sku_ids = df_transactions[~df_transactions['sku_id'].isin(df_catalog['id'])]['sku_id'].unique()

if invalid_sku_ids.any():
    print(red_color + "Invalid SKU IDs found in transactions:" + reset_color)
    print(invalid_sku_ids)

    invalid_sku_count = len(invalid_sku_ids)
    total_sku_count = len(df_transactions['sku_id'].unique())

    print(red_color + 'Total SKUs: {}'.format(total_sku_count) + reset_color)
    print(red_color + 'Number of invalid SKUs: {}'.format(invalid_sku_count) + reset_color)

    invalid_sku_percentage = (invalid_sku_count / total_sku_count) * 100
    print(red_color + 'Percentage of invalid SKUs from total SKUs: {:.2f}%'.format(invalid_sku_percentage) + reset_color)

else:
    print(green_color + "All SKU IDs in transactions exist in the df_catalog" + reset_color)

Invalid SKU IDs found in transactions:
['88850000' 'FU22-10000' 'FU22J-10000' ... '5052-01-0001' '22RMHA0009'
 'LIV10363']
Total SKUs: 22062
Number of invalid SKUs: 2153
Percentage of invalid SKUs from total SKUs: 9.76%


### SKU_id exist in the Source location OR Target location

Check if the SKU_id - target_location_id or SKU_id - source_location_id combinations exists in the df_inventory, if both of them are missing - raise an error.

In [50]:
inventory_sku_location_combinations = set(zip(df_inventory['sku_id'], df_inventory['location_id']))

missing_combinations = []

for _, row in df_transactions.iterrows():
    sku = row['sku_id']
    target_location = row['target_location_id']
    source_location = row['source_location_id']
    
    if (sku, target_location) not in inventory_sku_location_combinations and (sku, source_location) not in inventory_sku_location_combinations:
        missing_combinations.append((sku, target_location, source_location))

if len(missing_combinations) > 0:
    missing_df = pd.DataFrame(missing_combinations, columns=['sku_id', 'source_location_id', 'target_location_id'])
    print(red_color + "SKUs for which both SKU_id - target_location_id and SKU_id - source_location_id don't exist in df_inventory:" + reset_color)
    print(missing_df)
else:
    print(green_color + "All SKUs have either SKU_id - target_location_id or SKU_id - source_location_id combinations in df_inventory" + reset_color)

SKUs for which both SKU_id - target_location_id and SKU_id - source_location_id don't exist in df_inventory:
            sku_id source_location_id target_location_id
0              UR1                501                  1
1              UR1                502                  4
2              UR1                503                  6
3              UR1                504                  7
4              UR1                505                  8
...            ...                ...                ...
1043555  921519KSH            1049071                  9
1043556  921519KSH            1049072                  9
1043557  921519KSH            1049073                  9
1043558  921519KSH            1049074                  9
1043559  921519KSH            1049075                  9

[1043560 rows x 3 columns]
